In [1]:
# ! convert -delay X gym-results/openaigym.video.0.5889.video000000.mp4 -loop 0 cartpole.gif

<img src="gif/cartpole.gif" alt="CartPole" style="width: 500px;"/>

In [2]:
! python -m baselines.run --alg=deepq --env=CartPole-v0 --save_path=./cartpole_model.pkl --num_timesteps=1e5

2020-12-19 08:13:40.845761: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library cudart64_110.dll
Traceback (most recent call last):
  File "c:\users\cd_bo\appdata\local\programs\python\python38\lib\runpy.py", line 192, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\users\cd_bo\appdata\local\programs\python\python38\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "c:\work\openai\baselines\baselines\run.py", line 12, in <module>
    from baselines.common.cmd_util import common_arg_parser, parse_unknown_args, make_vec_env, make_env
  File "c:\work\openai\baselines\baselines\common\cmd_util.py", line 12, in <module>
    from gym.wrappers import FlattenDictWrapper
ImportError: cannot import name 'FlattenDictWrapper' from 'gym.wrappers' (C:\work\vpython\lib\site-packages\gym\wrappers\__init__.py)


In [3]:
# Load the model saved in cartpole_model.pkl and visualize the learned policy
! python -m baselines.run --alg=deepq --env=CartPole-v0 --load_path=./cartpole_model.pkl --num_timesteps=0 --play

2020-12-19 08:13:45.598700: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library cudart64_110.dll
Traceback (most recent call last):
  File "c:\users\cd_bo\appdata\local\programs\python\python38\lib\runpy.py", line 192, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\users\cd_bo\appdata\local\programs\python\python38\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "c:\work\openai\baselines\baselines\run.py", line 12, in <module>
    from baselines.common.cmd_util import common_arg_parser, parse_unknown_args, make_vec_env, make_env
  File "c:\work\openai\baselines\baselines\common\cmd_util.py", line 12, in <module>
    from gym.wrappers import FlattenDictWrapper
ImportError: cannot import name 'FlattenDictWrapper' from 'gym.wrappers' (C:\work\vpython\lib\site-packages\gym\wrappers\__init__.py)


In [4]:
import gym
import tensorflow as tf
# Import the desired algorithm from baselines
from baselines import deepq
from baselines import logger

In [ ]:
def callback(locals, globals):
    """
    function called at every step with state of the algorithm.
    If callback returns true training stops.
    """
    # stop training if average reward exceeds 199
    # time should be greater than 100 and the average of last 100 returns should be >= 199
    episode_rewards = locals["episode_rewards"]
    is_solved = (
        locals["t"] > 100 and sum(episode_rewards[-101:-1]) / 100 >= 195
    )
    if (is_solved):
        logger.record_tabular("solved :")
        mean_100ep_reward = round(np.mean(episode_rewards[-101:-1]), 1)
        num_episodes = len(episode_rewards)
        logger.record_tabular("steps", locals["t"])
        logger.record_tabular("episodes", num_episodes)
        logger.record_tabular("mean last 100 episode reward", mean_100ep_reward)
        logger.dump_tabular()

    return is_solved

# create the environment
env = gym.make("CartPole-v0")

# Prepare learning parameters: network and learning rate
# the policy is a multi-layer perceptron
network = "mlp"
# set learning rate of the algorithm
learning_rate = 5e-3

# launch learning on this environment using DQN
# ignore the exploration parameter for now

actor = deepq.learn(
    env,
    network=network,
    lr=learning_rate,
    total_timesteps=int(5e5),
    buffer_size=int(5e4),
    exploration_fraction=0.1,
    exploration_final_eps=0.02,
    print_freq=100,
    callback=callback
)

print("Saving model to cartpole_model.pkl")
save_path = "cartpole_model.pkl"
ckpt = tf.train.Checkpoint(model=actor)
manager = tf.train.CheckpointManager(ckpt, save_path, max_to_keep=None)
manager.save()
print("Saved to cartpole_model.pkl")
#actor.save("cartpole_model.pkl")

##%%

actor = deepq.learn(
    env,
    network=network,
    lr=learning_rate,
    total_timesteps=0,
    buffer_size=0,
    exploration_fraction=0.1,
    exploration_final_eps=0.02,
    print_freq=10,
    callback=callback,
    load_path="cartpole_model.pkl"
)

##%%

# Needed to show the environment in a notebook
from gym import wrappers
import numpy as np
from baselines.common.vec_env.vec_env import VecEnv
##%%

env = wrappers.Monitor(
    env, "./gym-results", force=True, video_callable=lambda episode_id: True
)

#visualize the policy
n_episodes = 5
n_timesteps = 1000
for episode in range(n_episodes):
    observation = env.reset()
    episode_return = 0
    for timestep in range(n_timesteps):
        # render the environment
        env.render()

        # select the action according to the actor
        if not isinstance(env, VecEnv):
            observation = np.expand_dims(np.array(observation), axis=0)
        action, _, _, _ = actor.step(tf.constant(observation))
        action = action[0].numpy()
        #action = actor(observation[None])[0]

        # call env.step function
        observation, reward, done, _ = env.step(action)

        # since the reward is undiscounted we can simply add the reward to the cumulated return
        episode_return += reward

        if done:
            break
    # here an episode is terminated, print the return and the number of steps
    print(f"Episode return {episode_return}, Number of steps: {timestep}")
env.close()

##%%

# Render the episodes
import io
import base64
from IPython.display import HTML, display
import ffmpy3
def render(episodes_to_watch=1):
    for episode in range(episodes_to_watch):
        videoPath = f"./gym-results/openaigym.video.{env.file_infix}.video{episode:06d}.mp4"
        videoPathWebm = videoPath.replace("mp4", "webm")
        ffmpy3.FFmpeg(
        inputs={videoPath: None},
        outputs={videoPathWebm: None}
        ).run()
        videoMp4 = io.open(videoPath, "r+b").read()
        videoWebm = io.open(videoPathWebm, "r+b").read()
        encodedMp4 = base64.b64encode(videoMp4)
        encodedWebm = base64.b64encode(videoWebm)
        display(
            HTML(
                data="""<video width="360" height="auto" alt="test" controls autoplay loop><source src="data:video/webm;base64,{1}" type="video/webm" /><source src="data:video/mp4;base64,{0}" type="video/mp4" /></video>""".format(
                    encodedMp4.decode("ascii"),encodedWebm.decode("ascii")
                )
            )
        )

render(5)


C:\work\vpython\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
C:\work\vpython\lib\site-packages\numpy\core\fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\work\vpython\lib\site-packages\numpy\core\_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


input shape is (4,)
input shape is (4,)
Logging to C:\Users\cd_bo\AppData\Local\Temp\openai-2020-12-19-08-13-59-931587
-------------------------------------------
| % time spent exploring       | 95       |
| episodes                     | 100      |
| mean last 100 episode reward | 22.5     |
| steps                        | 2228     |
-------------------------------------------
-------------------------------------------
| % time spent exploring       | 91       |
| episodes                     | 200      |
| mean last 100 episode reward | 21.2     |
| steps                        | 4348     |
-------------------------------------------
-------------------------------------------
| % time spent exploring       | 85       |
| episodes                     | 300      |
| mean last 100 episode reward | 28.7     |
| steps                        | 7219     |
-------------------------------------------
-------------------------------------------
| % time spent exploring       | 81       |
|